# Evaluate a SageMaker JumpStart model with FMeval and track with MLflow

***
Developed and tested on Jupyterlab App on Amazon SageMaker Studio, SageMaker Distribution 2.1.0, instance `ml.m5.2xlarge`
***

This notebook shows you how to use FMeval to evaluate a LLM deployed via SageMaker Jumpstart and track the evaluations as metrics with MLflow tracking server.

## Setup

### Import libraries

In [2]:
!pip uninstall torch torchvision -y

Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
Found existing installation: torchvision 0.19.1
Uninstalling torchvision-0.19.1:
  Successfully uninstalled torchvision-0.19.1


In [3]:
!pip install torch torchvision

  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl (766.7 MB)
Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl (7.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.2 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
autogluon-multimodal 1.2 requires scikit-learn<1.5.3,>=1.4.0, but you have scikit-learn 1.6.1 which is incompati

In [5]:
!pip uninstall torchvision -y
!pip uninstall torch -y
!pip uninstall fmeval -y

Found existing installation: torchvision 0.21.0
Uninstalling torchvision-0.21.0:
  Successfully uninstalled torchvision-0.21.0
Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
Found existing installation: fmeval 1.2.2
Uninstalling fmeval-1.2.2:
  Successfully uninstalled fmeval-1.2.2


In [6]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu
!pip install fmeval

Looking in indexes: https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 MB 72.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.2 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
autogluon-multimodal 1.2 requires scikit-learn<1.5.3,>=1.4.0, but you have scikit-learn 1.6.1 which is incompatible.
autogluon-multimodal 1.2 requires torch<2.6,>=2.2, but you have torch 2.6.0+cpu

In [1]:
from pathlib import Path

import mlflow
from dotenv import load_dotenv
from fmeval.constants import MIME_TYPE_JSONLINES
from fmeval.data_loaders.data_config import DataConfig
from fmeval.eval_algorithms.factual_knowledge import (
    FactualKnowledge,
    FactualKnowledgeConfig,
)
from fmeval.eval_algorithms.summarization_accuracy import SummarizationAccuracy
from fmeval.eval_algorithms.toxicity import Toxicity, ToxicityConfig
from fmeval.model_runners.sm_jumpstart_model_runner import JumpStartModelRunner
from utils import EvaluationSet, run_evaluation_sets, run_evaluation_sets_nested

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


[03/29/25 23:19:02] INFO     PyTorch version 2.6.0+cpu available.                                      ]8;id=894981;file:///opt/conda/lib/python3.11/site-packages/datasets/config.py\config.py]8;;\:]8;id=692860;file:///opt/conda/lib/python3.11/site-packages/datasets/config.py#54\54]8;;\

                    INFO     TensorFlow version 2.17.0 available.                                     ]8;id=336435;file:///opt/conda/lib/python3.11/site-packages/datasets/config.py\config.py]8;;\:]8;id=244634;file:///opt/conda/lib/python3.11/site-packages/datasets/config.py#112\112]8;;\

2025-03-29 23:19:05.753021: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
%load_ext autoreload
%autoreload 2

We set the environmental variables `MLFLOW_TRACKING_URI` and `MLFLOW_TRACKING_USERNAME` from the `.env` file created in [00-Setup](./00-Setup.ipynb).
Alternatively you can set the tracking URL using the `mlflow` sdk method:

``` python
mlflow.set_tracking_uri(tracking_server_arn)
```

In [3]:
load_dotenv()

True

Deploy the SageMaker Jumpstart endpoint you want to test. You need the corresponding `model_id` in SageMaker Jumpstart. It can be found when navigating in SageMaker Studio to the JumpStart section and looking at the model details or the sample notebook associated with the deployment section.

![jumpstart-model-id](../img/find-jumpstart-model-id.png)

Alternatively, if you have an existing SageMaker Jumpstart endpoint, you can replace the cell below by setting only the `endpoint_name` variable

```python
endpoint_name = "jumpstart-existing-endpoint-name"
```

In [7]:
# from sagemaker.jumpstart.model import JumpStartModel

# model_id = "hf-llm-mistral-7b-ins-20250329-220013Endpoint"  # e.g., "huggingface-llm-falcon2-11b"
# model = JumpStartModel(model_id=model_id)
# accept_eula = False  # <-- some Jumpstart models requires explicitly accepting a EULA
# predictor = model.deploy(accept_eula=accept_eula)
# endpoint_name = predictor.endpoint_name

ValueError: Invalid model ID: 'hf-llm-mistral-7b-ins-20250329-220013Endpoint'. Please visit https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html for a list of valid model IDs. The module `sagemaker.jumpstart.notebook_utils` contains utilities for fetching model IDs. We recommend upgrading to the latest version of sagemaker to get access to the most models.

In [4]:
endpoint_name = "jumpstart-dft-hf-llm-mistral-7b-ins-20250329-230644"

### Model Runner Setup

The model runner we create below will be used to perform inference on every sample in the dataset.

In [5]:
import json

from sagemaker.jumpstart.model import JumpStartModel
from sagemaker.jumpstart.session_utils import get_model_info_from_endpoint
from sagemaker.predictor import retrieve_default

Lets extract information about the model. One particularly important information is the `inputs` format, which tells us the prompt signature for the model we have deployed.

In [6]:
model_id, model_version, _, _, _ = get_model_info_from_endpoint(
    endpoint_name=endpoint_name
)
model = JumpStartModel(model_id=model_id, model_version=model_version)
predictor = retrieve_default(endpoint_name=endpoint_name)
sample_payload = model.retrieve_example_payload().body
print(json.dumps(sample_payload, indent=4))

No instance type selected for inference hosting endpoint. Defaulting to ml.g5.12xlarge.


[03/29/25 23:19:23] INFO     No instance type selected for inference hosting endpoint. Defaulting to   ]8;id=437932;file:///opt/conda/lib/python3.11/site-packages/sagemaker/jumpstart/factory/model.py\model.py]8;;\:]8;id=606576;file:///opt/conda/lib/python3.11/site-packages/sagemaker/jumpstart/factory/model.py#238\238]8;;\
                             ml.g5.12xlarge.                                                                       

{
    "inputs": "<s>[INST] what is the recipe of mayonnaise? [/INST]",
    "parameters": {
        "max_new_tokens": 256,
        "do_sample": true,
        "decoder_input_details": true,
        "details": true
    }
}


In [7]:
print(json.dumps(predictor.predict(sample_payload), indent=4))

[
    {
        "generated_text": "<s>[INST] what is the recipe of mayonnaise? [/INST]Mayonnaise is a creamy sauce made from oil, egg yolks, and an acid such as vinegar or lemon juice. Here's a basic recipe for traditional mayonnaise:\n\nIngredients:\n- 1 egg yolk\n- 1 teaspoon of mustard (Dijon mustard is a good choice)\n- 1 tablespoon of vinegar (white or apple cider vinegar) or lemon juice\n- 1 cup of oil (neutral oil like canola or vegetable oil work well)\n- Salt, to taste\n- Pepper, to taste\n\nInstructions:\n1. Combine the egg yolk, mustard, and acid (vinegar or lemon juice) in a medium-sized bowl. Whisk until well combined.\n2. Start adding the oil very slowly, drop by drop, while continually whisking the mixture. This process is crucial for keeping the sauce from separating.\n3. Once the sauce begins to thicken, you can start adding the oil in a thin, steady stream while continuing to whisk.\n4. Once all the oil is incorporated, season with salt and pepper to taste.\n5. If the

For JumpStart model, `FMeval`  gets payload and output formats from the description of the models, this make it easier to setup the runners. 

In [8]:
model_runner = JumpStartModelRunner(
    endpoint_name=endpoint_name,
    model_id=model_id,
    model_version=model_version,
)

Lets test our model runner. You should build the prompt according to the expected input signature of the model

In [9]:
model_runner.predict(prompt="What's the tallest building in the world?")

('What\'s the tallest building in the world? Well, that answer\'s actually pretty tricky depending on your definition of "tallest."\n\nWhen the Burj Khalifa in Dubai, UAE, first opened in 2010, it dwarfed every other construction, with 163 floors and 828 meters (2,700 feet) in height. (The shaft of the building is 828 meters to the very pointy top, but most buildings\' height measurements only account for the first 153 meters from the ground to the top-most part of the roof.)\n\nSadly, the United Arab Emirates seems to have a monopoly on the world\'s tallest buildings, keeping four out of the top five places. (The rest of the top 10 is a list of just-shy-of-tallest: One World Trade Center in New York is in 6th place, 85 Skyscraper in Taipei is in 5th, the Makkah Royal Clock Tower in Mecca is 3rd, and CTF Finans Tower in Guangzhou, China, is 4th.)\n\nBut the real',
 -23.06406305725)

### Data
We first check that the dataset file to be used by the evaluation is present, and then create a `DataConfig` object for each dataset. Each dataset has been prepared to evaluate one of the three categories, i.e., `Summarization`, `Factual Knowledge`, and `Toxicity`. More categories can be defined too.

In [10]:
dataset_path = Path("datasets")

dataset_uri_summarization = dataset_path / "gigaword_sample.jsonl"
if not dataset_uri_summarization.is_file():
    print("ERROR - please make sure the file, gigaword_sample.jsonl, exists.")

data_config_summarization = DataConfig(
    dataset_name="gigaword_sample",
    dataset_uri=dataset_uri_summarization.as_posix(),
    dataset_mime_type=MIME_TYPE_JSONLINES,
    model_input_location="document",
    target_output_location="summary",
)

dataset_uri_factual_knowledge = dataset_path / "trex_sample.jsonl"
if not dataset_uri_factual_knowledge.is_file():
    print("ERROR - please make sure the file, trex_sample.jsonl, exists.")

data_config_factual_knowledge = DataConfig(
    dataset_name="trex_sample",
    dataset_uri=dataset_uri_factual_knowledge.as_posix(),
    dataset_mime_type=MIME_TYPE_JSONLINES,
    model_input_location="question",
    target_output_location="answers",
)

dataset_uri_toxicity = dataset_path / "real_toxicity_sample.jsonl"
if not dataset_uri_toxicity.is_file():
    print("ERROR - please make sure the file, real_toxicity_sample.jsonl, exists.")

data_config_toxicity = DataConfig(
    dataset_name="toxicity_sample",
    dataset_uri=dataset_uri_toxicity.as_posix(),
    dataset_mime_type=MIME_TYPE_JSONLINES,
    model_input_location="prompt",
)

### Evaluation sets
We can now create an evaluation set for each algorithm we want to use in our test.

More information about metrics and evaluation sets can be found in the paper ["Evaluating Large Language Models with <i>fmeval</i>"](https://arxiv.org/pdf/2407.12872)

### Summarization

For the `Summarization` evaluation set, let's replace with your own prompt according to the input signature identified above. `FMeval` uses `$model_input` as placeholder to get the input from your evaluation dataset.

In [11]:
summarization_prompt = "Summarize the following text in one sentence: $model_input"

summarization_accuracy = SummarizationAccuracy()

evaluation_set_summarization = EvaluationSet(
    data_config_summarization,
    summarization_accuracy,
    summarization_prompt,
)

[nltk_data] Downloading package wordnet to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Toxicity

Lets do the same for Toxicity.

In [12]:
toxicity_prompt = "Complete the following sentence: $model_input"

toxicity = Toxicity(ToxicityConfig("detoxify"))

evaluation_set_toxicity = EvaluationSet(
    data_config_toxicity,
    toxicity,
    toxicity_prompt,
)

### Factual Knowledge

And again for Factual Knowledge.

In [13]:
factual_knowledge_prompt = "$model_input"

factual_knowledge = FactualKnowledge(
    FactualKnowledgeConfig(target_output_delimiter="<OR>")
)

evaluation_set_factual = EvaluationSet(
    data_config_factual_knowledge,
    factual_knowledge,
    factual_knowledge_prompt,
)

Group all evaluations

In [14]:
evaluation_list = [
    evaluation_set_summarization,
    evaluation_set_factual,
    evaluation_set_toxicity,
]

## Run evaluation

We setup the MLflow experiment used to track the evaluations.
We will then create a new run for each model, and run all the evaluation for that model within that run, so that the metrics will all appear together.  

We'll use the `model_id` as run name to make it easier to identify this run as part of the larger experiment, and run the evaluation using the `run_evaluation_sets()` defined in [utils.py](utils.py#20).

In [15]:
run_name = f"{model_id}"

In [16]:
experiment_name = "fmeval-mlflow-simple-runs"
experiment = mlflow.set_experiment(experiment_name)

In [17]:
with mlflow.start_run(run_name=run_name) as run:
    run_evaluation_sets(model_runner, evaluation_list)

2025-03-29 23:20:22,345	INFO worker.py:1812 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
[nltk_data] Downloading package wordnet to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2025-03-29 23:20:24,213	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-29_23-20-21_071722_922/logs/ray-data
2025-03-29 23:20:24,213	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMap

Running 0: 0.00 row [00:00, ? row/s]

- ReadCustomJSON->SplitBlocks(96) 1: 0.00 row [00:00, ? row/s]

- AggregateNumRows 2: 0.00 row [00:00, ? row/s]

2025-03-29 23:20:24,905	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-29_23-20-21_071722_922/logs/ray-data
2025-03-29 23:20:24,905	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCustomJSON]


...

[03/29/25 23:26:36] INFO     Saving dataset to file took 0.10 seconds.                              ]8;id=927481;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py\perf_util.py]8;;\:]8;id=924330;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py#18\18]8;;\

                    INFO     ===================================================                    ]8;id=373399;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py\perf_util.py]8;;\:]8;id=673934;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py#19\19]8;;\

                    INFO     Computing score and aggregation on dataset toxicity_sample took 64.78  ]8;id=404730;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py\perf_util.py]8;;\:]8;id=792659;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py#18\18]8;;\
                             seconds.                                                                              

                    INFO     ===================================================                    ]8;id=198216;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py\perf_util.py]8;;\:]8;id=869277;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py#19\19]8;;\

/opt/conda/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'datasets/real_toxicity_sample.jsonl'. Exception: 
  return _dataset_source_registry.resolve(
/opt/conda/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/opt/conda/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '/tmp/eval_results/toxicity_toxicity_sample.jsonl'. Exception: 
  return _dataset_source_registry.resolve(
/opt/conda/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: The spe

### Nested runs
An alternative approach to organize the runs is to create nested runs for the different tasks.

In [18]:
experiment_name = "fmeval-mlflow-nested-runs"
experiment = mlflow.set_experiment(experiment_name)

2025/03/29 23:30:27 INFO mlflow.tracking.fluent: Experiment with name 'fmeval-mlflow-nested-runs' does not exist. Creating a new experiment.


In [19]:
with mlflow.start_run(run_name=run_name, nested=True) as run:
    run_evaluation_sets_nested(model_runner, evaluation_list)

[nltk_data] Downloading package wordnet to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2025-03-29 23:30:29,398	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-29_23-20-21_071722_922/logs/ray-data
2025-03-29 23:30:29,399	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCustomJSON] -> AggregateNumRows[AggregateNumRows]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCustomJSON->SplitBlocks(96) 1: 0.00 row [00:00, ? row/s]

- AggregateNumRows 2: 0.00 row [00:00, ? row/s]

2025-03-29 23:30:29,518	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-29_23-20-21_071722_922/logs/ray-data
2025-03-29 23:30:29,519	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCustomJSON]


...

                    INFO     ===================================================                    ]8;id=488950;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py\perf_util.py]8;;\:]8;id=575749;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py#19\19]8;;\

                    INFO     Computing score and aggregation on dataset toxicity_sample took 67.33  ]8;id=627323;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py\perf_util.py]8;;\:]8;id=620585;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py#18\18]8;;\
                             seconds.                                                                              

                    INFO     ===================================================                    ]8;id=764654;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py\perf_util.py]8;;\:]8;id=636123;file:///opt/conda/lib/python3.11/site-packages/fmeval/perf_util.py#19\19]8;;\

MLflow parent run ID: 72ee6a618d3a4f3d8e6676fdc97415e7


/opt/conda/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'datasets/real_toxicity_sample.jsonl'. Exception: 
  return _dataset_source_registry.resolve(
/opt/conda/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/opt/conda/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '/tmp/eval_results/toxicity_toxicity_sample.jsonl'. Exception: 
  return _dataset_source_registry.resolve(
/opt/conda/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: The spe

The evaluation is completed, and the results are recorded in the MLflow tracking server.

To continue with the evaluation, you can move to the [compare_models.ipynb](./compare_models.ipynb)

## Clean up
Since SageMaker endpoints are [priced](https://aws.amazon.com/sagemaker/pricing/) by deployed infrastructure time rather than by requests, you can avoid unnecessary charges by deleting your endpoints when you're done experimenting.

[Here](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-delete-resources.html) you can find instructions on how to delete a SageMaker endpoint.